<a href="https://colab.research.google.com/github/AmarnadhReddy055/Jigsaw-Multilabel-Toxic-Comment-Classification/blob/main/Mutli_label_using_Neural_Networks_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import keras

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Toxic_comments_classification/final_toxic.csv')

In [ ]:
data.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,1,000103f0d9cfb60f,daww match background colour im seemingly stuc...,0,0,0,0,0,0
2,2,000113f07ec002fd,hey man im really trying edit war guy constant...,0,0,0,0,0,0
3,3,0001b41b1c6bb37e,cant make real suggestion improvement wondered...,0,0,0,0,0,0
4,4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0


In [ ]:
data.shape

(159571, 9)

In [ ]:
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

(159449, 9)

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping
from keras import regularizers

In [ ]:
from sklearn import model_selection

In [ ]:
X_train,X_test,Y_train,Y_test=model_selection.train_test_split(data['comment_text'],data.iloc[:,3:].values,test_size=0.4)

In [ ]:
max_features = 10000 # Number of words to keep
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_train.values)
test_sequences = tokenizer.texts_to_sequences(X_test.values)

In [ ]:
max_length = 100 # Maximum length of sequences
X_train = pad_sequences(train_sequences, maxlen=max_length)
X_test = pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    baseline=None,
    restore_best_weights=True
)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_length))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_data=[X_test, Y_test],callbacks=[early_stopping],use_multiprocessing=True)

Epoch 1/10
1495/1495 [==============================] - 109s 68ms/step - loss: 0.0753 - val_loss: 0.0519
Epoch 2/10
1495/1495 [==============================] - 29s 19ms/step - loss: 0.0483 - val_loss: 0.0506
Epoch 3/10
1495/1495 [==============================] - 20s 14ms/step - loss: 0.0428 - val_loss: 0.0523
Epoch 4/10
1495/1495 [==============================] - 23s 15ms/step - loss: 0.0379 - val_loss: 0.0543
Epoch 5/10
1495/1495 [==============================] - 18s 12ms/step - loss: 0.0339 - val_loss: 0.0589
Epoch 6/10
1495/1495 [==============================] - 20s 13ms/step - loss: 0.0296 - val_loss: 0.0650
Epoch 7/10
1495/1495 [==============================] - 16s 11ms/step - loss: 0.0259 - val_loss: 0.0740


In [ ]:
Y_pred=model.predict(X_test)

1994/1994 [==============================] - 6s 3ms/step


In [ ]:
y_pred = np.round(Y_pred)

In [ ]:
print(classification_report(y_pred,Y_test,target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],zero_division=0))

               precision    recall  f1-score   support

        toxic       0.70      0.87      0.78      5012
 severe_toxic       0.14      0.69      0.24       131
      obscene       0.76      0.85      0.80      3045
       threat       0.00      0.00      0.00         0
       insult       0.65      0.74      0.69      2804
identity_hate       0.00      0.00      0.00         0

    micro avg       0.64      0.83      0.73     10992
    macro avg       0.38      0.53      0.42     10992
 weighted avg       0.70      0.83      0.76     10992
  samples avg       0.06      0.06      0.06     10992



In [ ]:
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lemmatizer=WordNetLemmatizer()
stop_words=stopwords.words('english')

In [ ]:
def predict_label(text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = pad_sequences(sequence, maxlen=max_length)
  toxicity = model.predict(sequence)
  labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
  for i, label in enumerate(labels):
    print(f"{label}: {toxicity[0][i]}")

In [ ]:
predict_label('I hate you, you worthless piece of garbage')

1/1 [==============================] - 0s 25ms/step
toxic: 0.8838444352149963
severe_toxic: 0.020482752472162247
obscene: 0.20142975449562073
threat: 0.043366439640522
insult: 0.5048630833625793
identity_hate: 0.08346269279718399
